<a href="https://colab.research.google.com/github/chandyalex/GSoC-2021-TF-DarkPose/blob/main/notebooks/Dark_pose_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## About the colab notebook
This colab notebook demonstrating how to integrate the new pose estimation model with DARK pose tensorflow  library, 
- Since coco datset is large we choose the cpu colab for this demonsration.( GPU colab has not enough disk space to accomodate the entire data)
- The TPU support is still under development, final version of the library which handle all kind of hardware will be realease as the part of post GSoC project goals.

#### Step 1 
 - Making directory and downloading coco data set in zip format.
 - Unzipping downloded files .
 - Removing the zip file aftre extraction.

In [ ]:
# making directory and downloading data in to the disk
! mkdir -p /content/data/coco/images/
! wget http://images.cocodataset.org/zips/train2017.zip -P /content/data/coco/images
! wget http://images.cocodataset.org/zips/val2017.zip -P /content/data/coco/images
! wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P /content/data/coco
! unzip /content/data/coco/images/train2017.zip -d /content/data/coco/images
! unzip /content/data/coco/images/val2017.zip -d /content/data/coco/images
! unzip /content/data/coco/annotations_trainval2017.zip -d /content/data/coco/
! rm -r /content/data/coco/images/val2017.zip
! rm -r /content/data/coco/images/train2017.zip
! rm -r /content/data/coco/annotations_trainval2017.zip





#### Step 2
Installing all the dependencied for the library
- Cloning COCO API from git and installing
- Cloning Dark Pose Tensorflow repository and installing dependencies mentioned in 'requirements.txt'
- Building 'nms' library using make command inside the repository.

In [ ]:
# installing dependencies 

! git clone https://github.com/cocodataset/cocoapi.git
! git clone https://github.com/chandyalex/GSoC-2021-TF-DarkPose.git
! python /content/cocoapi/PythonAPI/setup.py install
! pip install -r /content/GSoC-2021-TF-DarkPose/requirements.txt
# building nms library
! cd /content/GSoC-2021-TF-DarkPose/  &&\make

#### Step 3
Importing all necessary libraries

In [ ]:
# importing essential libraries
import sys
sys.path.append('/content/GSoC-2021-TF-DarkPose')
import tensorflow as tf
from models import pose_resnet
from utils.utils import get_optimizer

from data import coco
from data import coco_data
from config import cfg
from config import update_config
from core.evaluation import accuracy
from core.dark_function import train,validate
from core.loss import JointsMSELoss,JointsOHKMMSELoss,dice_loss



#### Step 4
Setting up all the data directory and related pameters inside the config file.
Feel free to edit the config file in /experiments folder.

In [ ]:
# setting the data directory in to config
cfg.DATA_DIR="/content/data/coco/"
cfg.OUTPUT_DIR='out/'
cfg.LOG_DIR='log/'
cfg.DATASET.ROOT="/content/data/coco/"
cfg.DATASET.TEST_SET="val2017"
cfg.DATASET.TRAIN_SET="train2017"

#### Step 5
Loading training and validation dataset.

In [ ]:
# data loader
train_data = coco.COCODataset(cfg, cfg.DATASET.ROOT, cfg.DATASET.TRAIN_SET, True)
valid_dataset = coco.COCODataset(cfg, cfg.DATASET.ROOT, cfg.DATASET.TEST_SET, False)

loading annotations into memory...
Done (t=8.52s)
creating index...
index created!
loading annotations into memory...
Done (t=10.40s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
loading annotations into memory...
Done (t=1.46s)
creating index...
index created!


#### Step 6
At this point the pose estimation model can be initialized and build using input shape of data loader, in this colab we are using posresnet(baseline) model.

In [ ]:
# building model
model= pose_resnet.get_pose_net(
        cfg, is_train=True
    )

optimizer = get_optimizer(cfg, model)

model.build(input_shape=train_data[0][0].shape)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


#### Step 7
While training make sure that the train and validate function from 'core.dark_function'  library is used. The initilized model will be passed to the train function and estimation of heat map using DARK pose method will be takenplace inside these function. For detailed understating check the train.py script in the main repository.

In [ ]:
# training
epoch=1
for i in range(epoch):
      train(cfg,train_data,model,JointsMSELoss,optimizer,i)
      validate(cfg,valid_dataset,model,JointsMSELoss)

Model: "pose_res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  9472      
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
sequential_1 (Sequential)    (32, 32, 24, 256)         218624    
_________________________________________________________________
sequential_3 (Sequential)    (32, 16, 12, 512)         1226752   
_________________________________________________________________
sequential_5 (Sequential)    (32, 8, 6, 1024)         